In [ ]:
import os
os.environ["GEMINI_API_KEY"] = "your_api_key_here"

In [58]:
%pip install -q langchain-google-genai langchain requests langchain-core

Note: you may need to restart the kernel to use updated packages.


In [59]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [60]:
#tool create

from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetches the currency conversion factor from base_currency to target_currency."""

    url = f"https://v6.exchangerate-api.com/v6/b553a3885fbcc84b2d96fd3d/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """given a currency conversion rate and base currency, it returns the converted currency"""
    return base_currency * conversion_rate
    

In [61]:
get_conversion_factor({"base_currency": "USD", "target_currency": "INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1759363202,
 'time_last_update_utc': 'Thu, 02 Oct 2025 00:00:02 +0000',
 'time_next_update_unix': 1759449602,
 'time_next_update_utc': 'Fri, 03 Oct 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.7311}

In [62]:
convert.invoke({"base_currency": 100, "conversion_rate": 82.5})

8250.0

In [63]:
# tool binding

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [64]:
messages = [HumanMessage("What is the conversion rate from USD to INR? Convert 100 USD to INR")]

In [65]:
messages

[HumanMessage(content='What is the conversion rate from USD to INR? Convert 100 USD to INR', additional_kwargs={}, response_metadata={})]

In [66]:
ai_message = llm_with_tools.invoke(messages)

In [67]:
messages.append(ai_message)

In [68]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': 'd9f6bca0-f321-40d0-a0d4-72c4fcd05a2a',
  'type': 'tool_call'}]

In [69]:
import json

for tool_call in ai_message.tool_calls:
    #execute the first tool and get the value of conversion rate
    if tool_call['name'] == "get_conversion_factor":
        tool_message1 = get_conversion_factor.invoke(tool_call)
        print(tool_message1)
        #fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # append this tool message to the messages list
        messages.append(tool_message1)

    if tool_call['name'] == "convert":
        #fetch the current arg
        tool_call['arguments']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)


content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1759363202, "time_last_update_utc": "Thu, 02 Oct 2025 00:00:02 +0000", "time_next_update_unix": 1759449602, "time_next_update_utc": "Fri, 03 Oct 2025 00:00:02 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 88.7311}' name='get_conversion_factor' tool_call_id='d9f6bca0-f321-40d0-a0d4-72c4fcd05a2a'


In [70]:
messages

[HumanMessage(content='What is the conversion rate from USD to INR? Convert 100 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "INR", "base_currency": "USD"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f24330d1-b03d-44c4-8465-bfdde7971dab-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'INR', 'base_currency': 'USD'}, 'id': 'd9f6bca0-f321-40d0-a0d4-72c4fcd05a2a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 136, 'output_tokens': 347, 'total_tokens': 483, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 321}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-

In [71]:
llm_with_tools.invoke(messages).content

'The conversion rate from USD to INR is 88.7311.\n100 USD is equal to 8873.11 INR.'